## Setup

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error , mean_squared_error , mean_absolute_percentage_error

## The weather dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
zip_path = '/content/drive/MyDrive/appliances+energy+prediction.zip'
extract_path = '/content/drive/MyDrive/extracted_folder'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

csv_path = '/content/drive/MyDrive/extracted_folder/energydata_complete.csv'
df = pd.read_csv(csv_path)
temperature_series = df['Appliances']
date_time = pd.to_datetime(df.pop('date'), format='%Y-%m-%d %H:%M:%S')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


## Splitting of Data in 70:20:10 for ( training , validation , testing ) sets

In [ ]:
n = len(temperature_series)
# Split the data into training and testing sets

train , val , test = temperature_series[:int(n * (70/100))], temperature_series[int(n * (70/100)):int(n * (90/100))], temperature_series[int(n * (90/100)):]

## Normalize the data

In [ ]:
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
val = (val - train_mean) / train_std
test = (test - train_mean) / train_std
train.size

13814

In [ ]:
# Build and fit the ARIMA model to the training data
order = (5, 1, 0)  # You can experiment with different values for p, d, and q
arima = ARIMA(train, order=order)
model_fit = arima.fit()

In [ ]:
# Make predictions on the test set
predictions = model_fit.forecast(steps=len(test))

In [ ]:
# Evaluate the model's performance
mse = mean_squared_error(test, predictions)
mape = mean_absolute_percentage_error(test,predictions)
mae = mean_absolute_error(test,predictions);
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 0.975518134441991
Mean Absolute Percentage Error (MAPE): 4.669059937242509
Mean Absolute Error (MAE): 0.8059426586925283
